In [52]:
# Imports
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords

## Read and clean data

The code looks for a *data.txt* file exported directly from WhatsApp. Open a chat with a contact, go to the menu on the top right (three dots). Expand *More* and click *Export Chat*. You can export it without media since the code only needs the text file.

In [ ]:
datafile = 'data.txt'
data = pd.read_csv(datafile, sep='\n', header=None)
data.columns = ['text']
data.head()

In [ ]:
clean_data = data.copy()

# Construct regular expression
regex = re.compile('\d*\/\d*\/\d*,\s\d*:\d*\s[AP][M]\s-\s[A-Za-z\d\s]*:\s')
clean_data.apply(lambda x: re.sub(regex, '', x))